In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import os
import math

In [2]:
class CSVDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size=8, output_size=3, nr_hidden_layers=10, nr_neurons=128, activation=nn.LeakyReLU()):
        super(NeuralNetwork, self).__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        self.nr_hidden_layers = nr_hidden_layers
        self.nr_neurons = nr_neurons
        self.activation = activation
        
        # Input layer
        neurons = 2 ** math.ceil(math.log2(self.input_size))
        self.input_layer = nn.Linear(input_size, neurons)

        # Expanding layers
        self.expanding_layers = nn.ModuleList()
        while neurons < self.nr_neurons:
            self.expanding_layers.append(nn.Linear(neurons, neurons * 2))
            neurons *= 2

        # Hidden layers
        self.hidden_layers = nn.ModuleList([nn.Linear(neurons, neurons) for _ in range(nr_hidden_layers)])

        # Contracting layers
        self.contracting_layers = nn.ModuleList()
        while neurons > self.output_size*2:
            self.contracting_layers.append(nn.Linear(neurons, neurons // 2))
            neurons //= 2

        # Output layer
        self.output_layer = nn.Linear(neurons, output_size)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.activation(x)

        # Expanding layers
        for layer in self.expanding_layers:
            x = layer(x)
            x = self.activation(x)

        # Hidden layers
        for layer in self.hidden_layers:
            x = layer(x)
            x = self.activation(x)

        # Contracting layers
        for layer in self.contracting_layers:
            x = layer(x)
            x = self.activation(x)

        # Output layer
        x = self.output_layer(x)
        x = self.activation(x)
        
        return x

In [8]:
load_data('../Daten/Clean_Results_Isotherm.csv')

,Flow_well,Temp_diff,Temp_diff_real,kW_well,Hydr_gradient,Hydr_conductivity,Aqu_thickness,Long_dispersivity,Trans_dispersivity,Iso_width,Iso_distance,Area,Isotherm
0,1.7,3,2.338460,0.247,0.0015,0.00021,4,5,2.50,16.216287,28.171819,405.071857,1
1,4.3,3,2.909554,0.624,0.0015,0.00021,4,5,2.50,51.654721,210.434828,8951.648177,1
2,4.3,3,2.909554,0.624,0.0015,0.00021,4,5,2.50,16.640301,25.395053,393.286838,2
3,6.8,3,3.072444,0.987,0.0015,0.00021,4,5,2.50,85.651033,550.269109,38313.620295,1
4,6.8,3,3.072444,0.987,0.0015,0.00021,4,5,2.50,32.519463,77.868377,2181.234182,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85526,26394.7,7,4.725644,8938.761,0.0079,0.05800,30,35,1.75,10.856018,23.703290,218.756803,4
85527,28778.6,7,4.866222,9746.086,0.0079,0.05800,30,35,1.75,77.423866,1207.373768,75784.532769,1
85528,28778.6,7,4.866222,9746.086,0.0079,0.05800,30,35,1.75,35.360825,269.960083,7865.175621,2
85529,28778.6,7,4.866222,9746.086,0.0079,0.05800,30,35,1.75,21.122874,96.672952,1660.621827,3


,Flow_well,Temp_diff,Temp_diff_real,kW_well,Hydr_gradient,Hydr_conductivity,Aqu_thickness,Long_dispersivity,Trans_dispersivity,Iso_width,Iso_distance,Area,Isotherm
3114,0.3,7,3.100641,0.102,0.0015,0.00021,4,5,0.25,1.319414,1.573709,1.912099,2
3113,0.3,7,3.100641,0.102,0.0015,0.00021,4,5,0.25,5.959279,11.800293,63.892104,1
3239,0.4,7,2.776146,0.135,0.0015,0.00021,4,10,0.50,7.841949,18.713175,131.551303,1
3240,0.4,7,2.776146,0.135,0.0015,0.00021,4,10,0.50,1.754573,2.292542,3.712230,2
6640,0.5,7,3.260416,0.169,0.0015,0.00021,6,5,0.25,7.048497,15.053408,94.922025,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82074,312602.9,3,2.991050,45370.838,0.0079,0.05800,30,20,10.00,164.333468,665.260999,88751.390021,2
60671,333419.7,3,2.941919,48392.165,0.0060,0.05800,30,35,17.50,229.437274,750.529458,141933.383255,2
82098,338089.8,3,2.870309,49069.978,0.0079,0.05800,30,35,17.50,158.496143,399.554518,53980.067138,2
78531,370409.1,3,2.939043,53760.765,0.0079,0.05800,25,35,17.50,226.027649,733.108108,136740.243307,2


(array([[-2.5869851 ,  0.86003803, -2.14045568, ..., -1.18067199,
         -1.29813853, -0.18371664],
        [-2.5869851 ,  0.86003803, -2.14045568, ..., -1.18067199,
         -1.29813853, -1.28766615],
        [-2.55312958,  0.86003803, -2.12650917, ..., -0.27007948,
         -1.03841667, -1.28766615],
        ...,
        [ 0.90257211,  0.86003803,  1.0366521 , ...,  0.70134283,
         -0.05101024,  1.20709525],
        [ 0.90257211,  0.86003803,  1.0366521 , ...,  0.70134283,
         -0.05101024, -1.28766615],
        [ 0.90257211,  0.86003803,  1.0366521 , ...,  0.70134283,
         -0.05101024,  1.20709525]], shape=(68424, 9)),
 array([[ 0.90257211,  0.86003803,  1.0366521 , ...,  0.70134283,
         -0.05101024,  0.599548  ],
        [ 0.90257211,  0.86003803,  1.0366521 , ...,  0.70134283,
         -0.05101024,  0.599548  ],
        [ 0.90257211,  0.86003803,  1.0366521 , ...,  0.70134283,
         -0.05101024, -0.18371664],
        ...,
        [ 3.10922526, -1.90348948,  

In [ ]:
def load_data(csv_file,labels=[9]):
    # Load data
    df = pd.read_csv(csv_file)    
    df = df.sort_values(by='Flow_well')
    df = df.head(80000)    
    # Split features and labels
    X = df.iloc[:, [0,1,3,4,5,6,7,8,12] ].values  # Ignore column with real Temperature and use isotherm temperature
    y = df.iloc[:, labels].values # Three outputs to predict
    
    os.makedirs(f"plots/%s"%str(labels), exist_ok=True)
    
    for label in range(len(labels)):
        plt.hist(y[:,label],bins=200)
    plt.title("Before Transformation")
    plt.savefig(f"plots/%s/before.png"%str(labels))
    plt.close()
    
    X = np.log1p(X)
    y = np.log1p(y)
    
    for label in range(len(labels)):
        plt.hist(y[:,label],bins=200)
    plt.title("After Log-Transformation")
    plt.savefig(f"plots/%s/afterlog.png"%str(labels))
    plt.close()   
    
    # Standardize features
    X_scaler = StandardScaler()
    y_scaler = StandardScaler()
    
    X = X_scaler.fit_transform(X)
    y = y_scaler.fit_transform(y)
    
    for label in range(len(labels)):
        plt.hist(y[:,label],bins=200)
    plt.title("After Standardization")
    plt.savefig(f"plots/%s/afterstandardization.png"%str(labels))
    plt.close()
        
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
    return X_train, X_test, X_scaler, y_train, y_test, y_scaler


In [ ]:
def plot_results(x_loss,y_loss,true_values,predictions,labels):
    
    plt.plot(x_loss, y_loss)
    plt.title("Loss")
    plt.savefig(f"plots/%s/loss.png"%str(labels))
    plt.close()
    
    for label in range(len(labels)):
        true_values_label = true_values[:,label]
        predictions_label = predictions[:,label]
        plt.scatter(true_values_label, predictions_label, label=label)
        plt.plot([min(true_values_label), max(true_values_label)], [min(true_values_label), max(true_values_label)], 'r--')    
        plt.xlabel("True Values")
        plt.ylabel("Predicted Values")
        text = str(labels)
        plt.title("plot of labels:" + text)
        plt.legend()
        plt.savefig(f"plots/%s/truepred.png"%str(labels))

In [ ]:
def main(csv_file='../Daten/Clean_Results_Isotherm.csv', num_epochs=100, batch_size=64, learning_rate=0.01,labels=[9,10,11],nr_hidden_layers=3,nr_neurons=16):
    
    # Loss plot
    x_loss = np.linspace(start=1, stop=num_epochs,num=num_epochs)
    y_loss = [] 
    
    # Load data
    X_train, X_test, X_scaler, y_train, y_test, y_scaler = load_data(csv_file,labels=labels)
    
    # Convert to PyTorch datasets
    train_dataset = CSVDataset(X_train, y_train)
    test_dataset = CSVDataset(X_test, y_test)
        
    # Data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize the model
    input_size = X_train.shape[1]
    output_size = y_train.shape[1]
    
    model = NeuralNetwork(input_size, output_size, nr_hidden_layers=nr_hidden_layers, nr_neurons=nr_neurons)
    criterion = nn.MSELoss()  
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    print(model)
    summary(model, input_size = (1, input_size))
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for batch_data, batch_labels in train_loader:
            # Forward pass
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
        y_loss.append(loss.item())
    
    # Evaluate on the test set
    model.eval()
    with torch.no_grad():
        total_loss = 0
        predictions = []
        true_values = []
        for batch_data, batch_labels in test_loader: #direkt test daten vergleichen mit labels
            outputs = model(batch_data)
            predictions.extend(outputs.squeeze().tolist())
            true_values.extend(batch_labels.tolist())
            loss = criterion(outputs.squeeze(), batch_labels.squeeze())
            total_loss += loss.item()
    
    print(f"Test Loss: {total_loss / len(test_loader):.4f}")
    
    predictions = np.array(predictions).reshape(-1,len(labels))
    true_values = np.array(true_values).reshape(-1,len(labels))
                                      
    inverted_predictions = y_scaler.inverse_transform(predictions)
    inverted_true_values = y_scaler.inverse_transform(true_values)
    
    inverted_predictions = np.expm1(inverted_predictions)
    inverted_true_values = np.expm1(inverted_true_values)
    
    plot_results(x_loss,y_loss,inverted_true_values,inverted_predictions,labels)
    
    return inverted_predictions, inverted_true_values

In [ ]:
#predall, valall= main('../Daten/Clean_Results_Isotherm.csv',labels=[9,10,11])
pred9, val9 = main('../Daten/Clean_Results_Isotherm.csv',labels=[9])
pred10, val10 = main('../Daten/Clean_Results_Isotherm.csv',labels=[10])
pred11, val11 = main('../Daten/Clean_Results_Isotherm.csv',labels=[11])

In [ ]:
print("Predictions, True Values")
for i in range(10):
    print(pred9[i],", ",val9[i])
print("Predictions, True Values")
for i in range(10):
    print(pred10[i],", ",val10[i])
print("Predictions, True Values")
for i in range(10):
    print(pred11[i],", ",val11[i])

In [ ]:

rel_error = 0
for i in range(len(pred9)):
    rel_error += np.abs((pred9[i] - val9[i]) / pred9[i])
rel_error /= len(pred9)
print("Relative Error for label 9: ",rel_error)

rel_error = 0
for i in range(len(pred10)):
    rel_error += np.abs((pred10[i] - val10[i]) / pred10[i])
rel_error /= len(pred10)
print("Relative Error for label 10: ",rel_error)

rel_error = 0
for i in range(len(pred11)):
    rel_error += np.abs((pred11[i] - val11[i]) / pred11[i])
rel_error /= len(pred11)
print("Relative Error for label 11: ",rel_error)